In [1]:
%env CUBLAS_WORKSPACE_CONFIG=:4096:8

env: CUBLAS_WORKSPACE_CONFIG=:4096:8


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, random_split, DataLoader
from torchvision import models, transforms
from PIL import Image
import numpy as np
from tqdm import tqdm
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchinfo import summary
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import glob
from torch.utils.data import DataLoader
import os
import cv2
import torch
from torch.utils.data import Dataset, DataLoader,Subset
from tqdm import tqdm
import json
import random
import math
import re
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,confusion_matrix
import time
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve, auc

In [3]:
def set_seed(seed: int):
    """Seed everything for reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # enforce deterministic algorithms (may slow things down)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # torch 2.x: fully deterministic
    if hasattr(torch, "use_deterministic_algorithms"):
        torch.use_deterministic_algorithms(True)
# os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
# choose your seed
seed_list = [3,5,11,1344,2506]
SEED = 3
set_seed(SEED)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# Load Data

In [5]:
train_df = pd.read_csv('train_df_encoded_80_20.csv')
train_df

,image_path,label
0,C:\Users\Anwesh\Desktop\Anwesh\Oralpathology D...,0
1,C:\Users\Anwesh\Desktop\Anwesh\Oralpathology D...,0
2,C:\Users\Anwesh\Desktop\Anwesh\Oralpathology D...,0
3,C:\Users\Anwesh\Desktop\Anwesh\Oralpathology D...,0
4,C:\Users\Anwesh\Desktop\Anwesh\Oralpathology D...,0
...,...,...
1970,C:\Users\Anwesh\Desktop\Anwesh\Oralpathology D...,0
1971,C:\Users\Anwesh\Desktop\Anwesh\Oralpathology D...,1
1972,C:\Users\Anwesh\Desktop\Anwesh\Oralpathology D...,1
1973,C:\Users\Anwesh\Desktop\Anwesh\Oralpathology D...,1


In [6]:
test_df = pd.read_csv('test_df_encoded_80_20.csv')
test_df

,index,image_path,label
0,611,C:\Users\Anwesh\Desktop\Anwesh\Oralpathology D...,0
1,2325,C:\Users\Anwesh\Desktop\Anwesh\Oralpathology D...,1
2,99,C:\Users\Anwesh\Desktop\Anwesh\Oralpathology D...,0
3,144,C:\Users\Anwesh\Desktop\Anwesh\Oralpathology D...,0
4,612,C:\Users\Anwesh\Desktop\Anwesh\Oralpathology D...,0
...,...,...,...
489,1216,C:\Users\Anwesh\Desktop\Anwesh\Oralpathology D...,0
490,2165,C:\Users\Anwesh\Desktop\Anwesh\Oralpathology D...,1
491,398,C:\Users\Anwesh\Desktop\Anwesh\Oralpathology D...,0
492,668,C:\Users\Anwesh\Desktop\Anwesh\Oralpathology D...,0


In [7]:
total_abnormal_count = (train_df['label'] == 1).sum() 
total_normal_count = (train_df['label'] == 0).sum()
total_samples = len(train_df)

# Print
print("Total abnormal images:", total_abnormal_count)
print("Total normal images:", total_normal_count)
print("Total samples:", total_samples)
# Inverse frequency
weight_normal = 1 / total_normal_count
weight_abnormal = 1 / total_abnormal_count

# Normalize
total_inv = weight_normal + weight_abnormal
weight_normal /= total_inv
weight_abnormal /= total_inv

# PyTorch tensor
class_weights = torch.tensor([weight_normal, weight_abnormal], dtype=torch.float32).to(device)
print(class_weights)
print('Weight Ratio:',class_weights[1]/class_weights[0])

Total abnormal images: 260
Total normal images: 1715
Total samples: 1975
tensor([0.1316, 0.8684], device='cuda:0')
Weight Ratio: tensor(6.5962, device='cuda:0')


In [8]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to fit most CNNs
    transforms.ToTensor(),          # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet mean/std
                         std=[0.229, 0.224, 0.225])
])

class ImageDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.loc[idx, 'image_path']
        label = self.df.loc[idx, 'label']

        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.long)


train_dataset = ImageDataset(train_df, transform=transform)
test_dataset = ImageDataset(test_df, transform=transform)


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

# Model

In [17]:
# --- Training for one epoch -----------------------------------
def train_epoch(model, loader, optimizer,criterion, device):
    
    model.train()
    total_loss, total_correct, total_samples = 0.0, 0, 0
    
    step_times = []
    epoch_start = time.time()
    
    for images, labels in tqdm(loader, desc='Train', leave=False):
        step_start = time.time()
        images, labels = images.to(device), labels.to(device).long()

        optimizer.zero_grad()
        outputs = model(images).float()                  # [batch, 2] raw logits
        loss = criterion(outputs, labels)        # CrossEntropyLoss
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * labels.size(0)
        preds = outputs.argmax(dim=1)
        total_correct += (preds == labels).sum().item()
        total_samples += labels.size(0)

        step_end = time.time()
        step_times.append(step_end - step_start)
        
    avg_loss = total_loss / total_samples
    avg_acc  = total_correct / total_samples

    epoch_end = time.time()
    epoch_time = epoch_end - epoch_start
    
    return avg_loss, avg_acc, epoch_time, step_times


def validate_epoch(model, loader,criterion, device):
    model.eval()
    total_loss, total_correct, total_samples = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in tqdm(loader, desc='Val', leave=False):
            images, labels = images.to(device), labels.to(device).long()
            outputs = model(images).float()
            loss = criterion(outputs, labels)

            total_loss += loss.item() * labels.size(0)
            preds = outputs.argmax(dim=1)
            total_correct += (preds == labels).sum().item()
            total_samples += labels.size(0)

    avg_loss = total_loss / total_samples
    avg_acc  = total_correct / total_samples
    return avg_loss, avg_acc


    
def plot_training_metrics(metrics_path, model_name=None, save_dir=None, dpi=500):
    """
    Plots training and validation metrics from a single training session.

    Args:
        metrics_path (str): Path to the JSON file containing training metrics.
        model_name (str, optional): Name of the model for the plot title. 
                                    If None, inferred from the filename.
        save_dir (str, optional): Directory to save the plot. 
                                  If None, saved in the same directory as the metrics file.
        dpi (int): Dots per inch for the saved plot image.
    """
    # --- Load JSON Metrics ---
    with open(metrics_path, "r") as f:
        metrics = json.load(f)

    train_loss = metrics["train_loss"]
    val_loss = metrics["val_loss"]
    train_acc = metrics["train_acc"]
    val_acc = metrics["val_acc"]
    epochs = range(1, len(train_loss) + 1)

    # --- Set Model Name and Save Directory if not provided ---
    if model_name is None:
        model_name = os.path.splitext(os.path.basename(metrics_path))[0].replace("_metrics", "")
    
    if save_dir is None:
        save_dir = os.path.dirname(metrics_path)

    os.makedirs(save_dir, exist_ok=True)

    # --- Create Plot --- 📊
    fig, ax1 = plt.subplots(figsize=(12, 7))

    # Plot Loss on the primary y-axis
    l1 = ax1.plot(epochs, train_loss, 'o-', label='Train Loss', color='tab:red')
    l2 = ax1.plot(epochs, val_loss, 'o--', label='Validation Loss', color='tab:orange')
    ax1.set_xlabel('Epochs', fontsize=12)
    ax1.set_ylabel('Loss', color='tab:red', fontsize=12)
    ax1.tick_params(axis='y', labelcolor='tab:red')
    ax1.grid(True, which='both', linestyle='--', linewidth=0.5)

    # Create a secondary y-axis for Accuracy
    ax2 = ax1.twinx()
    l3 = ax2.plot(epochs, train_acc, 's-', label='Train Accuracy', color='tab:blue')
    l4 = ax2.plot(epochs, val_acc, 's--', label='Validation Accuracy', color='tab:cyan')
    ax2.set_ylabel('Accuracy', color='tab:blue', fontsize=12)
    ax2.tick_params(axis='y', labelcolor='tab:blue')

    # --- Final Touches (Title and Legend) ---
    plt.title(f'{model_name}: Training & Validation Metrics', fontsize=16, pad=40)
    
    # Combine legends from both axes into one
    lines = l1 + l2 + l3 + l4
    labels = [line.get_label() for line in lines]
    fig.legend(
        handles=lines,
        labels=labels,
        loc='upper center',
        bbox_to_anchor=(0.5, 1.02),
        ncol=4, # Display legend in one row
        fontsize='medium',
        frameon=True
    )
    
    fig.tight_layout(rect=[0, 0, 1, 0.96]) # Adjust layout to make space for the title

    # --- Save the Plot ---
    save_path = os.path.join(save_dir, f"{model_name}_training_metrics.png")
    plt.savefig(save_path, dpi=dpi, bbox_inches='tight')
    plt.close()

    print(f"✅ Saved plot: {save_path}")

In [18]:
def get_resnet50(name: str, num_classes=2, device='cuda'):
    base_model = models.resnet50
    # --- Pretrained or not ---
    if "scratch" in name:
        model = base_model(weights=None, zero_init_residual=True)
        in_features = model.fc.in_features
        model.fc = nn.Linear(in_features, num_classes)
        for param in model.parameters():
            param.requires_grad = True
        return model.to(device)

        
    else:
        model = base_model(weights='DEFAULT')
    # --- Modifications ---
    in_features = model.fc.in_features
    dropout_match = re.search(r'dp\(([\d.]+)\)', name)
    dropout_p = float(dropout_match.group(1)) if dropout_match else None


    
    if "mod2" in name:
        # Mod 2: Two-layer MLP with dropout
        model.fc = nn.Sequential(
            nn.Linear(in_features, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout_p if dropout_p is not None else 0.5),
            nn.Linear(256, num_classes)
        )
        for param in model.parameters():
            param.requires_grad = False
        for param in model.layer4[:].parameters():
            param.requires_grad = True
        for param in model.fc.parameters():
            param.requires_grad = True
            
    elif "mod1" in name:
        # Mod 1: Dropout + final layer
        model.fc = nn.Sequential(
            nn.Linear(in_features, in_features),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout_p if dropout_p is not None else 0.5),
            nn.Linear(in_features, num_classes)
                                )
        if "layer4_1" in name:
            for param in model.parameters():
                param.requires_grad = False
            for param in model.layer4[:].parameters():
                param.requires_grad = True
            for param in model.fc.parameters():
                param.requires_grad = True
        else:
            for param in model.parameters():
                param.requires_grad = False
            for param in model.layer4[:].parameters():
                param.requires_grad = True
            for param in model.fc.parameters():
                param.requires_grad = True
 
    else:
        # Default
        model.fc = nn.Linear(in_features, num_classes)
        for param in model.parameters():
            param.requires_grad = False
        for param in model.layer4[:].parameters():
            param.requires_grad = True
        for param in model.fc.parameters():
            param.requires_grad = True

    return model.to(device)

## Model Config

In [19]:
NUM_EPOCHS = 100
BATCH_SIZE = 32
BATCH_SIZE    = 32
EARLY_STOPING_PATIENCE = 20
INITIAL_LR = 0.001
REDUCE_LR_PATIENCE = 10
REDUCE_LR_FACTOR = 0.5
CLASS_WEIGHTS = class_weights

In [20]:
def run_training(
    model_name_prefix,
    train_loader = train_loader,
    val_loader = test_loader,
    get_model_fn = get_resnet50,
    train_epoch_fn = train_epoch,
    validate_epoch_fn = validate_epoch,
    plot_metrics_fn = plot_training_metrics,
    class_weights = CLASS_WEIGHTS,
    device='cuda',
    batch_size=BATCH_SIZE,
    num_epochs=NUM_EPOCHS,
    patience=EARLY_STOPING_PATIENCE,
    lr=INITIAL_LR,
    weight_decay=1e-5,
    seed=SEED,
    dir_path = os.getcwd(),
    metrics_filename=None
):

    model_dir = os.path.join(dir_path, model_name_prefix)
    os.makedirs(model_dir, exist_ok=True)


    all_metrics = []
    print('Training',model_name_prefix) 

    # Model and optimizer
    model = get_model_fn(model_name_prefix)
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=REDUCE_LR_PATIENCE)
    if 'weighted' in model_name_prefix:
        criterion = nn.CrossEntropyLoss(weight=class_weights.to(torch.float32))
        print('Using Weighted Loss')
    else:
        criterion = nn.CrossEntropyLoss()
    # History for this fold
    hist = {
        "train_loss": [], "train_acc": [],
        "val_loss": [],   "val_acc": [],
        "epoch_time_s": [], "avg_step_time_s": [],
        "lr_history": []
    }

    best_loss = float('inf')
    epochs_no_improve = 0

    model_basename = f"{model_name_prefix}"
    model_path = os.path.join(model_dir, f"{model_basename}.pth")
    best_model_path = os.path.join(model_dir, f"{model_basename}_best.pth")

    for epoch in range(1, num_epochs + 1):
        train_loss, train_acc, epoch_time, step_times = train_epoch_fn(model, train_loader, optimizer, criterion, device)
        val_loss, val_acc = validate_epoch_fn(model, val_loader, criterion, device)
        scheduler.step(val_loss)

        # Record
        hist["train_loss"].append(train_loss)
        hist["train_acc"].append(train_acc)
        hist["val_loss"].append(val_loss)
        hist["val_acc"].append(val_acc)
        hist["epoch_time_s"].append(epoch_time)
        hist["avg_step_time_s"].append(sum(step_times) / len(step_times))
        hist["lr_history"].append(optimizer.param_groups[0]['lr'])

        print(f"Epoch {epoch:02d}: Train Loss={train_loss:.4f}, Acc={train_acc*100:.2f}% | "
              f"Val Loss={val_loss:.4f}, Acc={val_acc*100:.2f}% | LR={hist['lr_history'][-1]:.6f}")

        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(model.state_dict(), best_model_path)
            epochs_no_improve = 0
            print("  → New best model saved")
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print(f"Early stopping at epoch {epoch}")
                break

    # Save final model for fold
    torch.save(model.state_dict(), model_path)
    all_metrics.append(hist)




    # Save to file
    if metrics_filename is None:
        metrics_filename = f"{model_name_prefix}_training_metrics.json"
    metrics_path = os.path.join(model_dir, metrics_filename)

    with open(metrics_path, "w") as f:
        json.dump(all_metrics[0], f, indent=4)
    
    print(f"\nK-fold cross-validation complete. Metrics written to {metrics_path}.")
    plot_metrics_fn(metrics_path=metrics_path)
    model_summary_str = str(summary(model, input_size=(32, 3, 224, 224)))
    summary_path = os.path.join(model_dir, f"{model_name_prefix}_summary.txt")
    with open(summary_path, "w", encoding="utf-8") as f:
        f.write(model_summary_str)

In [21]:
model_list = [
    # "ResNet18_scratch",
    "ResNet50_weighted_scratch",
    # "ResNet34",
    # "ResNet34_weighted",
    # "ResNet34_mod1",
    # "ResNet34_mod1_weighted",
    "ResNet50_mod2",
    "ResNet50_mod2_weighted",
    # "ResNet18_mod1_weighted_dp(0.2)",
    "ResNet50_mod1_weighted_dp(0.4)",
    # "ResNet18_mod1_weighted_dp(0.6)"
]

In [ ]:
for model_name in model_list:
    run_training(model_name)

Training ResNet18_mod2


Epoch 01: Train Loss=0.3009, Acc=87.49% | Val Loss=0.2246, Acc=90.69% | LR=0.001000
  → New best model saved


Epoch 02: Train Loss=0.1678, Acc=93.97% | Val Loss=0.3080, Acc=90.08% | LR=0.001000


Epoch 03: Train Loss=0.1000, Acc=95.95% | Val Loss=0.4525, Acc=89.47% | LR=0.001000


Train:  69%|████████████████████████████████████████████████████                       | 43/62 [00:16<00:07,  2.64it/s]